In [3]:
pip install spotipy

  Using cached spotipy-2.23.0-py3-none-any.whl (29 kB)
  Using cached redis-5.0.1-py3-none-any.whl (250 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
  Attempting uninstall: requests
    Found existing installation: requests 2.24.0
    Uninstalling requests-2.24.0:
      Successfully uninstalled requests-2.24.0
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Denish\anaconda3\python.exe -m pip install --upgrade pip' command.


In [23]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [24]:
secrets_file = open("secrets.txt","r")

In [25]:
string = secrets_file.read()

In [29]:
#string

In [28]:
#string.split('\n')

In [30]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [31]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

# Playlist 

In [32]:
# we will use our 'first paid music' playlist as an example:
playlist = sp.user_playlist_tracks("spotify", "1TM3rECdFTc0R0tgiIf0oW")

In [12]:
#playlist

In [13]:
playlist.keys() 

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [14]:
#playlist["items"][0]

In [15]:
playlist["total"]

4690

In [16]:
len(playlist['items'])

100

In [17]:
playlist['next']

'https://api.spotify.com/v1/playlists/1TM3rECdFTc0R0tgiIf0oW/tracks?offset=100&limit=100&additional_types=track'

In [18]:
#playlist['items']

In [19]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3000)/1000) # respectful nap
    return tracks

In [22]:
all_tracks = get_playlist_tracks("1TM3rECdFTc0R0tgiIf0oW")
len(all_tracks)

4690

In [ ]:
import pandas as pd
from pandas import json_normalize
tracks2 = json_normalize(all_tracks)

In [ ]:
tracks2.head()

In [ ]:
artists_for_song2 = tracks2.iloc[0]['track.id']
artists_for_song2 

In [ ]:
artists_df2 = pd.DataFrame(columns=['href', 'id', 'name', 'type', 'uri', 'external_urls.spotify','song_id', 'song_name', 'popularity' ])
for i in tracks2.index:
    artists_for_song = json_normalize(tracks2.iloc[i]['track.artists'])
    artists_for_song['song_id']    = tracks2.iloc[i]['track.id']         # we want to keep song_id, it is the sae for all artists
    artists_for_song['song_name']  = tracks2.iloc[i]['track.name']       # we want to keep song_name, it is the sae for all artists
    artists_for_song['popularity'] = tracks2.iloc[i]['track.popularity'] # same for popularity   
    artists_df2 = pd.concat([artists_df2, artists_for_song], axis=0)

In [ ]:
df_final2 = artists_df2[['song_name', 'name', 'song_id', 'popularity']]
df_final2

In [ ]:
df_final2.reset_index(drop = True)

In [ ]:
df_final2.dropna(inplace = True)

In [ ]:
# probably let them figure this out in the lab :-)
chunks = [(i, i+100) for i in range(0, len(df_final2), 100)]
chunks
audio_features_list = []
for chunk in chunks:
    id_list100 = df_final2['song_id'][chunk[0]:chunk[1]]
    audio_features_list = audio_features_list + sp.audio_features(id_list100)
    sleep(randint(1,3000)/1000)
len(audio_features_list)

In [ ]:
audio_features_df = json_normalize(audio_features_list)

In [ ]:
audio_features_df.drop_duplicates(inplace=True) 

In [ ]:
df_w_audio_ft = pd.merge(left=df_final2,
                        right=audio_features_df,
                        how='inner',
                        left_on='song_id',
                        right_on='id')
df_w_audio_ft

In [ ]:
df_w_audio_ft.to_csv("tejal_Audio_track.csv")

In [ ]:
tejal_Audio_track = pd.read_csv("tejal_Audio_track.csv")

In [ ]:
Katrz_Audio_track = pd.read_csv("playlist_katarzyna.csv")

In [ ]:
Katrz_Audio_track.shape

In [ ]:
tejal_Audio_track.shape

In [ ]:
final_data = pd.concat([tejal_Audio_track,Katrz_Audio_track],axis=0)

In [ ]:
final_data.drop_duplicates(inplace=True)